# Dual Grid Construction

A dual grid is a secondary grid built from grid, by constructing new faces over the original grids nodes. One property of this means that if you take the dual grid of a dual grid, the orignal grid will be constructed. However, this will only work properly if the grid is not partial. UXarray allows the constructing of this dual grid on all three of our data structures using the following methods:

* `grid.get_dual()`
* `dataarray.get_dual()`
* `dataset.get_dual()`

In [ ]:
import uxarray as ux
import warnings

warnings.filterwarnings("ignore")

plot_kwargs = {
    "backend": "matplotlib",
    "aspect": 2,
    "fig_size": 400,
}

In [ ]:
file = "../../test/meshfiles/mpas/QU/mesh.QU.1920km.151026.nc"

uxds = ux.open_dataset(file, file)

## Computing the Dual Grid

When computing the dual of a grid, the original grid is typically referred to as the "Primal" grid. The corner nodes of the Primal Grid become the face centers of the Dual Grid, and the face centers of the Primal Grid become the corner nodes of the Dual Grid. This means that variables that were originally face centered on the Primal Grid will be node centered on the Dual grid, and vice versa. Using UXarray we can construct this dual mesh, using `grid.compute_dual()` which returns a new grid object.

In [ ]:
grid = uxds.uxgrid

In [ ]:
dual = grid.get_dual()

In [ ]:
(
    grid.plot(title="Primal", backend="matplotlib", aspect=2)
    + dual.plot(title="Constructed Dual", backend="matplotlib", aspect=2)
).opts(fig_size=350).cols(1)

In [ ]:
grid.plot(title="Primal=Blue\nDual=Red", backend="bokeh") * dual.plot(
    backend="bokeh"
) * grid.plot.face_centers(backend="bokeh", size=5, color="black")

## Computing the Dual Grid with Data

We can also take a dual mesh of UxDataArrays. The concept for constructing the grid remains the same, and the grid constructed will be identical. The difference is the data stored inside the UxDataArray will be transfered with the dual mesh. The key differences is the location that the data is stored. The data transfer process works as follows:

* Face centered data becomes node centered, as each face becomes a node in the dual mesh.
* Node centered data becomes face centered, as each node becomes a face in the dual mesh.
* Edge centered data remains unchanged, as the edge centers will remain in the same place, despite the edges themselves being different.

### Face Centered

Constructing the dual mesh from a face centered variable, the data becomes node centered. We can then plot this using `topological_mean` to get the dual data to the faces for proper visualization comparisions.

In [ ]:
uxds_dual_face = uxds["latCell"].get_dual()

In [ ]:
(
    uxds["latCell"].plot.rasterize(
        backend="matplotlib",
        title="Face centered data on Primal Mesh",
        cmap=ux.cmaps.sequential_green_blue,
    )
    + uxds_dual_face.topological_mean(destination="face").plot.rasterize(
        backend="matplotlib",
        title="Node Centered Data on Dual Mesh",
        cmap=ux.cmaps.sequential_green_blue,
    )
).opts(fig_size=350).cols(1)

### Node Centered Data

Constructing the dual mesh from a node centered variable, the data becomes face centered. This time the primal mesh will be the one using the topological mean, as for visualization the data needs to be stored on the face centers.

In [ ]:
uxds_dual_node = uxds["xVertex"].get_dual()

In [ ]:
(
    uxds["xVertex"]
    .topological_mean(destination="face")
    .plot.rasterize(
        backend="matplotlib",
        title="Node centered data on Primal Mesh",
        cmap=ux.cmaps.sequential_green_blue,
    )
    + uxds_dual_node.plot.rasterize(
        backend="matplotlib",
        title="Face Centered Data on Dual Mesh",
        cmap=ux.cmaps.sequential_green_blue,
    )
).opts(fig_size=350).cols(1)

### Edge Centered Data

Constructing the dual mesh from a edge centered variable, the data stays on the same edge centers. However, a plotting example cannot be shown, as the `topological_mean` needed to visualize edge centered data is not currently implemented for edge centered data.

### UxDatasets

We can also construct a dual mesh from an entire dataset, which will convert the entire UxDataset to its dual mesh form. Below we can see the dataset before the dual mesh is constructed.

In [ ]:
uxds

Now we can construct the dual and see the new dataset that is returned.

In [ ]:
uxds_dual = uxds.get_dual()

In [ ]:
uxds_dual

As you can see, transforms the whole dataset. We can now take any variable and plot it, as shown below.

In [ ]:
uxds_dual["xVertex"].plot.rasterize(
    backend="matplotlib",
    title="xVertex from UxDataset dual mesh",
    cmap=ux.cmaps.sequential_green_blue,
).opts(fig_size=350)